# Live Event Monitor

Real-time WebSocket event monitoring with filtering and phase detection.

In [ ]:
import sys
from pathlib import Path
import time
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd

# Add lib to path
lib_path = Path.cwd().parent.parent / 'lib'
if str(lib_path) not in sys.path:
    sys.path.insert(0, str(lib_path))

from cdp_notebook import CDPCapture, MockCDPCapture

## 1. Connect

In [ ]:
USE_MOCK = False  # Set True for testing without Chrome

capture = MockCDPCapture() if USE_MOCK else CDPCapture()
capture.connect()

## 2. Event Filter Controls

In [ ]:
# Filter widgets
event_filter = widgets.Dropdown(
    options=['All', 'gameStateUpdate', 'playerUpdate', 'usernameStatus', 
             'playerJoined', 'playerLeft', 'chatMessage'],
    value='All',
    description='Event Type:'
)

limit_slider = widgets.IntSlider(
    value=20,
    min=5,
    max=100,
    description='Show:'
)

refresh_rate = widgets.Dropdown(
    options=[('1 second', 1), ('2 seconds', 2), ('5 seconds', 5)],
    value=2,
    description='Refresh:'
)

controls = widgets.HBox([event_filter, limit_slider, refresh_rate])
display(controls)

## 3. Live Monitor

In [ ]:
def detect_phase(event):
    """Detect game phase from event data."""
    data = event.get('data', {})
    
    if data.get('cooldownTimer', 0) > 0:
        return 'COOLDOWN'
    elif data.get('allowPreRoundBuys') and not data.get('active'):
        return 'PRESALE'
    elif data.get('active') and not data.get('rugged'):
        return 'ACTIVE'
    elif data.get('rugged'):
        return 'RUGGED'
    return '-'

def format_events(events, limit=20):
    """Format events for display."""
    if not events:
        return pd.DataFrame()
    
    rows = []
    for e in events[-limit:]:
        data = e.get('data', {})
        rows.append({
            'time': e.get('captured_at', '')[-12:-4],
            'event': e.get('event_name', ''),
            'phase': detect_phase(e),
            'price': f"{data.get('price', ''):.2f}" if 'price' in data else '-',
            'tick': data.get('tick', '-'),
        })
    
    return pd.DataFrame(rows)

In [ ]:
# Live monitoring loop
MONITOR_DURATION = 300  # 5 minutes

output = widgets.Output()
display(output)

start_time = time.time()

try:
    while (time.time() - start_time) < MONITOR_DURATION:
        with output:
            clear_output(wait=True)
            
            # Get filtered events
            filter_val = event_filter.value
            if filter_val == 'All':
                events = list(capture.events)
            else:
                events = capture.filter_events(filter_val)
            
            # Display stats
            elapsed = int(time.time() - start_time)
            print(f"Elapsed: {elapsed}s | Total events: {len(capture.events)} | Filtered: {len(events)}")
            print()
            
            # Display event table
            df = format_events(events, limit=limit_slider.value)
            if not df.empty:
                display(HTML(df.to_html(index=False)))
            else:
                print("Waiting for events...")
            
            # Show distribution
            print("\nEvent Distribution:")
            counts = capture.get_event_counts()
            for name, count in list(counts.items())[:5]:
                print(f"  {name}: {count}")
        
        time.sleep(refresh_rate.value)

except KeyboardInterrupt:
    print("\nMonitoring stopped by user")

print(f"\nFinal count: {len(capture.events)} events")

## 4. Export Session

In [ ]:
# Export all captured events
import json
from datetime import datetime

output_dir = Path.home() / 'rugs_recordings' / 'notebook_captures'
output_dir.mkdir(parents=True, exist_ok=True)

session_id = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = output_dir / f'monitor_session_{session_id}.jsonl'

events = capture.get_events()
with open(output_file, 'w') as f:
    for event in events:
        f.write(json.dumps(event) + '\n')

print(f"Exported {len(events)} events to:")
print(f"  {output_file}")

## 5. Cleanup

In [ ]:
capture.disconnect()

## Next Steps

- **03_batch_capture.ipynb** - Overnight batch capture for knowledge pipeline